<a href="https://colab.research.google.com/github/bmacmurray99/proof-of-concept/blob/main/proof_of_concept.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install tldextract cohere tiktoken openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [24]:
import os
import pandas as pd
import tldextract
import re
#Uses OpenAI calls for generic use
from openai import OpenAI

#TODO -  Add In Azure ML OpenAI

client = OpenAI(api_key = "OPEN_AI_API_KEY")

#Load in the data. In Azure ML platform, data stores can be accessed with a block of pre-generated code referencing workspace and loads data in one dataframe
#Since we"re working in a generic Python notebook, we"ll just load in from a given directory

#TODO -  add code from "Consume" in Azure ML Platform

  # Load all CSVs into one dataframe, similar end result to Azure ML code Loading

# This function is for data with dissimilar table structures that should be loaded as individual dataframes and returned as a dict (useful for ScreamingFrog output as it outputs a lot of data with varying amounts of columns/titles)
# Use when you need to reference
def dict_of_dataframes(directory):
  dataframes_dict = {}
  # Read each file in the directory and store in the dictionary
  for filename in os.listdir(directory):
      if filename.endswith(".csv"):
          filepath = os.path.join(directory, filename)
          # Read the file into a DataFrame and use the filename as the key in the dictionary
          dataframes_dict[filename] = pd.read_csv(filepath)
  return dataframes_dict



#Global URL manipulation functions, these are global because both both the PageOperationClass and BaseOperationClass need some URL manipulation
#TODO - add a base class for the following functions (BaseOperationsClass), abstract page_filter and keyword_filter into a generic funtion named 'filt' or whatever
#TODO -  add code to have PageOperationClass and BaseOperationClass inherit from BaseOperationClass

 #Class for machine learning calls
 #TODO -  Remove references to unassigned attributes, re-factor so that
class MachineLearningCalls:
#call OpenAI to get a best-match category for each keyword from a pre-defined list of categories

  def get_keyword_category(self,keyword):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {
        "role": "system",

        "content": f"You are a categorization bot that determines the most relevant category for a given word. You can only output a category from the ones defined for you, and responses other than those categories is a catastrophic error. Do not output a full sentence. You will use your strengths as an LLM to choose the best category for the keyword {keyword}. Your response should consist exclusively of one of the following categories: {self.categories}."
      }
    ],
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
    print(response.choices[0].message)
    return response.choices[0].message.content


  def detect_address(self,keyword):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {
        "role": "system",
        "content": f" You are a bot that detects whether or not a given phrase is related to a physical location. Your responses are limited to 'True  or 'False'. Do not output a full sentence, just 'True' or 'False'. Tell me if {keyword} is related to a physical location."
      }
    ],
    temperature=0.5,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
    print(response.choices[0].message)
    return response.choices[0].message.content










#Generic class for data manipulations specific to keywords - categorization, etc.

class BaseOperationClass(MachineLearningCalls):
  def __init__(self,df,categories=None,keyword_column="Keyword",url_column ="Address"):
    self.df = df
    self.categories = categories
    self.keyword_column = keyword_column
    self.url_column = url_column
#Keyword Column default is "Keyword"
#UNTESTED - No access to OpenAI API provided in AzureML by default, awaiting approval
#TODO -  Generate Categories from the keyword column/ a filtered slice of the keywords ranked by volume.

  def get_base_domain(self,url):
    ext = tldextract.extract(url)
    return f"{ext.domain}.{ext.suffix}"


  def get_subdomain(self,url):
    ext = tldextract.extract(url)
    return f"{ext.subdomain}.{ext.domain}.{ext.suffix}"

  #Make the OpenAI calls for each keyword
  def build_categories(self):
    self.df["Category"] = self.df[self.keyword_column].apply(self.get_keyword_category)

  def identify_address(self):
    self.df["Is Address"] =  self.df[self.keyword_column].apply(self.detect_address)

#Regular pandas functions, no remote stuff.

  def count_words(self):
    self.df["Word Count"] = self.df[self.keyword_column].apply(lambda x : len(x.split()))

  def page_filter(self,regex):
    return self.df[self.df[self.url_column].apply(lambda x : True if re.search(regex,x) else False)]

  def keyword_filter(self,regex):
    return self.df[self.df[self.keyword_column].apply(lambda x : True if re.search(regex,x) else False)]


#Class for analyzing Ahrefs keyword data
class AhrefsKeywords(BaseOperationClass):
  def __init__(self,df,categories=None):
    super().__init__(df,categories,url_column="Current URL")
    #Lets call all the applicable calculations we can do on the initial dataframe here
    self.count_words()
    self.difficulty_by_percentile()
    self.volume_by_percentile()
    self.df["Base Domain"] = self.df[self.url_column].apply(self.get_base_domain)
    self.df["Difficulty Percentile"] = self.df["KD"].rank(pct=True)
    self.df["Volume Percentile"] =  self.df["Volume"].rank(pct=True)

  def group_by_base_domain(self):
    grouped_data = self.df.groupby("Base Domain").agg({
        "Current URL": "nunique",
        self.keyword_column:"nunique",
        "Volume": "sum",
        "KD": "mean",
        "Traffic": "sum",
    })
    grouped_data.columns = ["Count of Current URL","Count of Keywords","Sum of Volume", "Average of KD", "Sum of Traffic"]
    return grouped_data

  #Group By categories generated from get_keyword_category
  def group_by_category(self):
    self.build_categories()
    grouped_categories =  self.df.groupby("Category").agg({
        "Current URL":"nunique",
        "Volume":"sum",
        "KD":"mean",
        "Traffic":"sum"
    })
    grouped_categories.columns = ["Count of Current URL","Sum of Volume","Average of KD","Sum of Traffic"]
    return grouped_categories


  def keyword_overlap(self):
    kw_overlap = self.df.groupby(self.keyword_column).agg({
        "Base Domain":"nunique",
        "Volume":"sum",
        "KD":"mean",
        "Traffic":"sum"
    })
    kw_overlap.columns = ["Count of Base Domain","Sum of Volume","Average of KD","Sum of Traffic"]
    return kw_overlap





  def difficulty_by_percentile(self):
    self.df["Difficulty Percentile"] = self.df["KD"].rank(pct=True)

  def volume_by_percentile(self):
    self.df["Volume Percentile"] =  self.df["Volume"].rank(pct=True)



# Class for Google Search Console Data
class SearchConsoleKeywords(BaseOperationClass):
  #Pass a new Keyword column if it needs to be renamed
  def __init__(self,df,categories=None,keyword_column ="Top queries"):
    super().__init__(df,categories,keyword_column)
    self.count_words()
    self.df["Impressions Percentile"] =  self.df["Impressions"].rank(pct=True)
    self.df["Positon Percentile"] =  self.df["Position"].rank(pct=True)
    self.df["Clicks Percentile"] =  self.df["Clicks"].rank(pct=True)

  def group_by_category(self):
    self.build_categories()
    grouped_categories =  self.df.groupby("Category").agg({
        self.keyword_column: "nunique",
        "Clicks":"sum",
        "Impressions":"sum",
        "Position": "mean"

    })
    grouped_categories.columns = ["Count of Keywords","Sum of Clicks", "Sum of Impressions", "Avg of Position" ]
    return grouped_categories


#Class for ScreamingFrog data
class ScreamingFrogPages(BaseOperationClass):
    def __init__(self,df):
      super().__init__(df)
      self.df["Subdomain"] = self.df[self.url_column].apply(self.get_subdomain)
    def group_by_subdomain(self):

      grouped_subdomains = self.df.groupby("Subdomain").agg({
          self.url_column: "nunique"

      })
      grouped_subdomains.columns = ["Unique Count of URLs"]
      return grouped_subdomains


class SearchConsolePages(BaseOperationClass):
  def __init__(self,df):
      super().__init__(df,url_column="Top pages")
      self.df["Impressions Percentile"] =  self.df["Impressions"].rank(pct=True)
      self.df["Positon Percentile"] =  self.df["Position"].rank(pct=True)
      self.df["Clicks Percentile"] =  self.df["Clicks"].rank(pct=True)
      self.df["Subdomain"] = self.df[self.url_column].apply(self.get_subdomain)


In [21]:
a = pd.read_csv("/content/sample_data/ahrefs_test_data.csv")
b = pd.read_csv("/content/sample_data/searchconsole-test.csv")
c = pd.read_csv("/content/sample_data/searchconsole-pages-test.csv")
d= pd.read_csv("/content/sample_data/internal_all.csv")
keyword_operation_class_a = BaseOperationClass(a)
ahrefs = AhrefsKeywords(a, categories=["Physical Security","Banking","Anti-Counterfeit Measures","Cybersecurity","Finance Operations"])
keyword_operation_class_b = BaseOperationClass(b)
gsc = SearchConsoleKeywords(b,categories= ["Netflix","Hulu","Movies","Streaming","Food"])
gscpage = SearchConsolePages(c)
screm = ScreamingFrogPages(d)
print(gscpage.page_filter(".*adding.*"))

print(ahrefs.url_column)
print(gsc.url_column)
print(screm.url_column)
print(gscpage.url_column)

print(ahrefs.keyword_column)
print(gsc.keyword_column)
print(screm.keyword_column)
print(gscpage.keyword_column)




                                           Top pages  Clicks  Impressions  \
0  https://bryanmacmurray.com/adding-netflix-to-s...     473        10556   

     CTR  Position  Impressions Percentile  Positon Percentile  \
0  4.48%     10.17                     1.0                0.75   

   Clicks Percentile            Subdomain  
0                1.0  .bryanmacmurray.com  
Current URL
Address
Address
Top pages
Keyword
Top queries
Keyword
Keyword


In [22]:
ahrefs.group_by_base_domain()

,Count of Current URL,Count of Keywords,Sum of Volume,Average of KD,Sum of Traffic
Base Domain,,,,,
accuweather.com,1,36,15800,29.277778,36
google.com,1,59,7490,16.050847,59
wikipedia.org,1,39,647500,41.717949,39


In [23]:
#And now, the demo exports

with pd.ExcelWriter("/content/sample_data/ahrefs_extract.xlsx") as writer:
  #Export all keywords
  ahrefs.df.to_excel(excel_writer = writer,sheet_name="All Keywords")
  #summarize keywords that overlap
  ahrefs.keyword_overlap().to_excel(excel_writer = writer,sheet_name="Keyword Overlap")
  #Filter for Location Pages
  ahrefs.page_filter(".*location/.*").to_excel(excel_writer = writer, sheet_name ="Location Pages")
  #Give Stats on the Base Domain Groups
  ahrefs.group_by_base_domain().to_excel(excel_writer = writer, sheet_name ="Domain Stats")
#Write the Google Search Console Data - Keywords and Pages.
with pd.ExcelWriter("/content/sample_data/search_console_extract.xlsx") as sc_writer:
  #Keywords
  gsc.df.to_excel(excel_writer = sc_writer,sheet_name="All Keywords")
  #Brand Keywords
  gsc.keyword_filter(".*(w|W)ellstar.*").to_excel(excel_writer = sc_writer, sheet_name = "Brand Keywords")
  #Top Keywords
  gsc.df[gsc.df["Clicks Percentile"] >= 0.75 ].to_excel(excel_writer = sc_writer, sheet_name ="Top KW By Clicks")
  #All Pages
  gscpage.df.to_excel(excel_writer = sc_writer, sheet_name="All Pages")
  #Location Pages
  gscpage.page_filter(".*location/.*").to_excel(excel_writer = writer, sheet_name ="Location Pages")
  #Top Pages (75+ Percentile)
  gscpage.df[gsc.df["Clicks Percentile"] >= 0.75 ].to_excel(excel_writer = sc_writer, sheet_name = "Top Pages By Clicks")

with pd.ExcelWriter("/content/sample_data/screamingfrog_extract.xlsx") as sf_writer:
  screm.df.to_excel(excel_writer = sf_writer, sheet_name = "All Pages")
  screm.page_filter(".*location/.*").to_excel(excel_writer = sf_writer, sheet_name ="Location Pages")
  screm.group_by_subdomain().to_excel(excel_writer = sf_writer, sheet_name ="Stats by Subdomain")



<ipython-input-23-e2baef77b148>:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gscpage.df[gsc.df["Clicks Percentile"] >= 0.75 ].to_excel(excel_writer = sc_writer, sheet_name = "Top Pages By Clicks")


In [ ]:
""